In [2]:
import openeo

# connection = openeo.connect("https://earthengine.openeo.org")
connection = openeo.connect("https://openeo.dataspace.copernicus.eu/openeo/1.2")

In [6]:
connection.authenticate_oidc_device()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=WDYP-QZOF 📋 to authenticate.

✅ Authorized successfully

<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2' with OidcBearerAuth>

In [9]:
connection

<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2' with OidcBearerAuth>

3 auth modes. Seems that with copernicus oicd need some request to admin.

In [10]:
# Now that we are connected, we can initialize our datacube object with the area around Vienna 
# and the time range of interest using Sentinel 1 data.
datacube = connection.load_collection("SENTINEL1_GRD",
    spatial_extent = {
        "west": -54.61,
        "south": 2.11,
        "east": -51.63,
        "north": 5.85
    },
    temporal_extent=["2017-03-01", "2017-06-01"],
    bands=["VV"]
)

In [11]:
datacube

In [13]:
# Since we are creating a monthly RGB composite, we need three (R, G and B) separated time ranges.
# Therefore, we split the datacube into three datacubes by filtering temporal for March, April and May. 
march = datacube.filter_temporal("2017-03-01", "2017-04-01")
april = datacube.filter_temporal("2017-04-01", "2017-05-01")
may = datacube.filter_temporal("2017-05-01", "2017-06-01")

# Now that we split it into the correct time range, we have to aggregate the timeseries values into a single image.
# Therefore, we make use of the Python Client function `mean_time`, which reduces the time dimension, 
# by taking for every timeseries the mean value.

mean_march = march.mean_time()
mean_april = april.mean_time()
mean_may = may.mean_time()

# Now the three images will be combined into the temporal composite. 
# Before merging them into one datacube, we need to rename the bands of the images, because otherwise, 
# they would be overwritten in the merging process.  
# Therefore, we rename the bands of the datacubes using the `rename_labels` process to "R", "G" and "B".
# After that we merge them into the "RGB" datacube, which has now three bands ("R", "G" and "B")

R_band = mean_march.rename_labels(dimension="bands", target=["R"])
G_band = mean_april.rename_labels(dimension="bands", target=["G"])
B_band = mean_may.rename_labels(dimension="bands", target=["B"])

RG = R_band.merge_cubes(G_band)
RGB = RG.merge_cubes(B_band)


# Last but not least, we add the process to save the result of the processing. There we define that 
# the result should be a GeoTiff file.
# We also set, which band should be used for "red", "green" and "blue" color in the options.

RGB = RGB.save_result(format="GTIFF")

In [14]:
# With the last process we have finished the datacube definition and can create and start the job at the back-end.

job = RGB.create_job()
job.start_and_wait().download_results()

0:00:00 Job 'j-2504181524044637a5ed4a070fa95f93': send 'start'
0:00:18 Job 'j-2504181524044637a5ed4a070fa95f93': created (progress 0%)
0:00:24 Job 'j-2504181524044637a5ed4a070fa95f93': created (progress 0%)
0:00:30 Job 'j-2504181524044637a5ed4a070fa95f93': created (progress 0%)
0:00:39 Job 'j-2504181524044637a5ed4a070fa95f93': created (progress 0%)
0:00:50 Job 'j-2504181524044637a5ed4a070fa95f93': created (progress 0%)
0:01:02 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:01:18 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:01:38 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:02:02 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:02:33 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:03:10 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:03:57 Job 'j-2504181524044637a5ed4a070fa95f93': running (progress N/A)
0:04:56 Job 'j-2504181524044637a5ed4a070fa95f93': running (progres

OpenEoApiError: [403] TokenInvalid: Authorization token has expired or is invalid. Please authenticate again. (ref: r-25042118550244a8ba110f3c66320a99)